## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 17th Nov. 24th


In [39]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [40]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [41]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [42]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [43]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-22-2024,Condo/Co-op,9441 SW 4th St #206,Miami,FL,33174.0,310000.0,2.0,2.0,VERSAILLE GARDENS I CONDO,1086.0,NaN,1975.0,NaN,285.0,390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9441-SW-4th-St...,MARMLS,A11638549,N,Y,25.765521,-80.350523


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [44]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [45]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
106,PAST SALE,2024-11-25,Condo/Co-op,8365 SW 152nd Ave Unit C-308,Miami,FL,33193.0,245000.0,2.0,1.0,VERABELLA FALLS CONDO,828.0,NaN,1984.0,NaN,296.0,234.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8365-SW-152nd-...,MARMLS,A11546114,N,Y,25.688575,-80.438462
1,PAST SALE,2024-11-22,Condo/Co-op,9441 SW 4th St #206,Miami,FL,33174.0,310000.0,2.0,2.0,VERSAILLE GARDENS I CONDO,1086.0,NaN,1975.0,NaN,285.0,390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9441-SW-4th-St...,MARMLS,A11638549,N,Y,25.765521,-80.350523
18,PAST SALE,2024-11-22,Condo/Co-op,7330 Ocean Ter Unit 15-D,Miami Beach,FL,33141.0,1106000.0,2.0,2.0,ST TROPEZ OCEAN CONDO,1410.0,NaN,1999.0,NaN,784.0,1437.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7330-Oce...,MARMLS,A11678982,N,Y,25.858999,-80.120145
81,PAST SALE,2024-11-22,Condo/Co-op,398 NE 5th St #1504,Miami,FL,33132.0,620000.0,1.0,1.0,#,443.0,NaN,2022.0,NaN,1400.0,506.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/398-NE-5th-St-...,MARMLS,A11631157,N,Y,25.778880,-80.189056
74,PAST SALE,2024-11-22,Condo/Co-op,8180 Geneva Way #420,Doral,FL,33166.0,320000.0,2.0,2.0,LAS VISTAS AT DORAL CONDO,992.0,NaN,1984.0,NaN,323.0,472.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/8180-Geneva-Ct...,MARMLS,A11635254,N,Y,25.817569,-80.329007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,PAST SALE,2024-11-18,Condo/Co-op,11113 Biscayne Blvd #2057,Miami,FL,33181.0,720000.0,3.0,2.5,JOCKEY CLUB III CONDO,2749.0,NaN,1982.0,NaN,262.0,2087.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11113-Biscayne...,MARMLS,A11656601,N,Y,25.879104,-80.165897
91,PAST SALE,2024-11-18,Condo/Co-op,253 NE 2nd St #1806,Miami,FL,33132.0,400000.0,1.0,1.0,VIZCAYNE SOUTH CONDO,806.0,NaN,2008.0,NaN,496.0,981.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/253-NE-2nd-St-...,MARMLS,A11598913,N,Y,25.776414,-80.189069
65,PAST SALE,2024-11-18,Condo/Co-op,8430 NW 8th St Unit B10,Miami,FL,33126.0,339000.0,2.0,2.0,MIDWAY FOREST CONDO,920.0,NaN,1984.0,NaN,368.0,400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8430-NW-8th-St...,MARMLS,A11614793,N,Y,25.777521,-80.332909
66,PAST SALE,2024-11-18,Condo/Co-op,10839 NW 7th St Unit 12-11,Miami,FL,33172.0,345000.0,3.0,2.0,TERRANOVA CONDO #3,1215.0,NaN,1983.0,NaN,284.0,375.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10839-NW-7th-S...,MARMLS,A11630668,N,Y,25.778064,-80.372267


In [46]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [47]:
df_top_ten = df.head(10)

## Collect Agent Information

In [48]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [49]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [50]:
response_df = pd.DataFrame(response_list)

In [51]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [52]:
df_top_ten = merged_df

## Current Week's Values

In [53]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

113


In [54]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$688,002


In [55]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$546


In [56]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$77,744,235


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
102
Input Previous Week Condo Average Sales Price:
1022796
Input Previous Week Condo Average PSF:
606
Input Previous Week Condo Sales total (ex: 198_000_000)
92_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [57]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [58]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [59]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [60]:
pd.set_option('display.max_columns',None)

In [61]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.12315,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange


In [62]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange
1,PAST SALE,2024-11-22,Condo/Co-op,1000 S Pointe Dr #2503,Miami Beach,FL,33139.0,3850000.0,3.0,2.5,MURANO AT PORTOFINO CONDO,2008.0,NaN,2002.0,NaN,1917.0,2782.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11643530,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Michelle Halfon,Sator Realty LLC,None,None,Laura Pantano,Coldwell Banker Realty,None,None,2,blue
2,PAST SALE,2024-11-19,Condo/Co-op,10201 Collins Ave #905,Bal Harbour,FL,33154.0,3450000.0,2.0,2.5,OCEANA BAL HARBOUR CONDO,1704.0,NaN,2016.0,NaN,2025.0,3655.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11649870,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Allen Davoudpour,"Compass Florida, LLC",None,None,None,None,None,None,3,blue
3,PAST SALE,2024-11-20,Condo/Co-op,1000 S Pointe Dr #705,Miami Beach,FL,33139.0,2600000.0,2.0,2.5,MURANO AT PORTOFINO,1407.0,NaN,2002.0,NaN,1848.0,1967.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11568588,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Lisa Van Wagenen,Brown Harris Stevens,Gabriela Pellegrini,Brown Harris Stevens,Jim Brinker,LoKation,None,None,4,blue
4,PAST SALE,2024-11-20,Condo/Co-op,296 Sea View Dr Unit C4,Key Biscayne,FL,33149.0,2160000.0,3.0,3.5,CAPE FLA CLUB CONDO CLUST,1800.0,NaN,1970.0,NaN,1200.0,1927.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,MARMLS,A11666942,N,Y,25.685372,-80.158768,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,Sandra Poli,United Realty Group Inc,None,None,Ignacio Villanueva,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-11-19,Condo/Co-op,4100 Island Blvd #2301,Aventura,FL,33160.0,1800000.0,3.0,3.0,BELLINI AT WILLIAMS ISLAN,2235.0,NaN,2013.0,NaN,805.0,2811.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/4100-Island...,MARMLS,A11608468,N,Y,25.941140,-80.135967,https://www.redfin.com/FL/Aventura/4100-Island...,Nina Vazquez,The Keyes Company,Alfredo Morejon,The Keyes Company,Joanne Perez,"Williams Island Realty, LLC",None,None,6,blue
6,PAST SALE,2024-11-18,Condo/Co-op,6000 Island Blvd #1804,Aventura,FL,33160.0,1800000.0,3.0,2.5,BELLAMARE @ WILLIAMS ISLA,2030.0,NaN,2005.0,NaN,887.0,2844.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11671839,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Karen Matluck,"Compass Florida, LLC",None,None,Richard Goihman,Douglas Elliman,None,None,7,blue
7,PAST SALE,2024-11-21,Condo/Co-op,7930 East Dr #1808,North Bay Village,FL,33141.0,1688000.0,5.0,3.5,ELOQUENCE ON THE BAY COND,2748.0,NaN,2009.0,NaN,614.0,2983.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11534963,N,Y,25.852669,-80.157847,https://www.redfin.com/FL/North-Bay-Village/79...,Sebastien Studer,Studer Realty LLC,None,None,Belkis Jorge,Brown Harris Stevens,None,None,8,blue
8,PAST SALE,2024-11-22,Condo/Co-op,13647 Deering Bay Dr #112,Coral Gables,FL,33158.0,1625000.0,3.0,3.5,DEERING BAY CONDO I,3140.0,NaN,1993.0,NaN,518.0,3539.0,Sold,NaN,NaN,https://www.redfin.com/FL/

In [65]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [66]:
m.save('index.html')

## Data snagger

In [67]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [68]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.12315,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_11252024


# CREATE TEMPLATE 

In [70]:
muni_set = set(df_top_ten['CITY'])

In [71]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [72]:
df_top_ten.reset_index(inplace=True,drop=True)

In [73]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [74]:
top_sale

'OCEANA BAL HARBOUR CONDO at 10201 Collins Ave #2406 in Bal Harbour'

In [75]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [76]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [77]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
1,PAST SALE,2024-11-22,Condo/Co-op,1000 S Pointe Dr #2503,Miami Beach,FL,33139.0,3850000.0,3.0,2.5,MURANO AT PORTOFINO CONDO,2008.0,NaN,2002.0,NaN,1917.0,2782.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11643530,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Michelle Halfon,Sator Realty LLC,None,None,Laura Pantano,Coldwell Banker Realty,None,None,2,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
2,PAST SALE,2024-11-19,Condo/Co-op,10201 Collins Ave #905,Bal Harbour,FL,33154.0,3450000.0,2.0,2.5,OCEANA BAL HARBOUR CONDO,1704.0,NaN,2016.0,NaN,2025.0,3655.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11649870,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Allen Davoudpour,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2024-11-20,Condo/Co-op,1000 S Pointe Dr #705,Miami Beach,FL,33139.0,2600000.0,2.0,2.5,MURANO AT PORTOFINO,1407.0,NaN,2002.0,NaN,1848.0,1967.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11568588,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Lisa Van Wagenen,Brown Harris Stevens,Gabriela Pellegrini,Brown Harris Stevens,Jim Brinker,LoKation,None,None,4,blue,MURANO AT PORTOFINO at 1000 S Pointe Dr #705 i...
4,PAST SALE,2024-11-20,Condo/Co-op,296 Sea View Dr Unit C4,Key Biscayne,FL,33149.0,2160000.0,3.0,3.5,CAPE FLA CLUB CONDO CLUST,1800.0,NaN,1970.0,NaN,1200.0,1927.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,MARMLS,A11666942,N,Y,25.685372,-80.158768,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,Sandra Poli,United Realty Group Inc,None,None,Ignacio Villanueva,Douglas Elliman,None,None,5,blue,CAPE FLA CLUB CONDO CLUST at 296 Sea View Dr U...
5,PAST SALE,2024-11-19,Condo/Co-op,4100 Island Blvd #2301,Aventura,FL,33160.0,1800000.0,3.0,3.0,BELLINI AT WILLIAMS ISLAN,2235.0,NaN,2013.0,NaN,805.0,2811.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/4100-Island...,MARMLS,A11608468,N,Y,25.941140,-80.135967,https://www.redfin.com/FL/Aventura/4100-Island...,Nina Vazquez,The Keyes Company,Alfredo Morejon,The Keyes Company,Joanne Perez,"Williams Island Realty, LLC",None,None,6,blue,BELLINI AT WILLIAMS ISLAN at 4100 Island Blvd ...
6,PAST SALE,2024-11-18,Condo/Co-op,6000 Island Blvd #1804,Aventura,FL,33160.0,1800000.0,3.0,2.5,BELLAMARE @ WILLIAMS ISLA,2030.0,NaN,2005.0,NaN,887.0,2844.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11671839,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Karen Matluck,"Compass Florida, LLC",None,None,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2024-11-21,Condo/Co-op,7930 East Dr #1808,North Bay Village,FL,33141.0,1688000.0,5.0,3.5,ELOQUENCE ON THE BAY COND,2748.0,NaN,2009.0,NaN,614.0,2983.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11534963,N,Y,25.8

In [78]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA BAL HARBOUR CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,600,000M to $6,562,500M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Coral Gables, Miami, Aventura, Miami Beach, North Bay Village, Bal Harbour, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 113 condo sales totaling $77,744,235 million from Nov. 17th to Nov. 24th. The previous week, brokers closed 102 condo sales totaling $92,500,000.

Last week’s units sold for an average of $688,002, lower than the $1,022,796 average price fro

In [79]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [80]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [81]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [82]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
1,PAST SALE,2024-11-22,Condo/Co-op,1000 S Pointe Dr #2503,Miami Beach,FL,33139.0,3850000.0,3.0,2.5,MURANO AT PORTOFINO CONDO,2008.0,NaN,2002.0,NaN,1917.0,2782.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11643530,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Michelle Halfon,Sator Realty LLC,None,None,Laura Pantano,Coldwell Banker Realty,None,None,2,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
2,PAST SALE,2024-11-19,Condo/Co-op,10201 Collins Ave #905,Bal Harbour,FL,33154.0,3450000.0,2.0,2.5,OCEANA BAL HARBOUR CONDO,1704.0,NaN,2016.0,NaN,2025.0,3655.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11649870,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Allen Davoudpour,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2024-11-20,Condo/Co-op,1000 S Pointe Dr #705,Miami Beach,FL,33139.0,2600000.0,2.0,2.5,MURANO AT PORTOFINO,1407.0,NaN,2002.0,NaN,1848.0,1967.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11568588,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Lisa Van Wagenen,Brown Harris Stevens,Gabriela Pellegrini,Brown Harris Stevens,Jim Brinker,LoKation,None,None,4,blue,MURANO AT PORTOFINO at 1000 S Pointe Dr #705 i...
4,PAST SALE,2024-11-20,Condo/Co-op,296 Sea View Dr Unit C4,Key Biscayne,FL,33149.0,2160000.0,3.0,3.5,CAPE FLA CLUB CONDO CLUST,1800.0,NaN,1970.0,NaN,1200.0,1927.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,MARMLS,A11666942,N,Y,25.685372,-80.158768,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,Sandra Poli,United Realty Group Inc,None,None,Ignacio Villanueva,Douglas Elliman,None,None,5,blue,CAPE FLA CLUB CONDO CLUST at 296 Sea View Dr U...
5,PAST SALE,2024-11-19,Condo/Co-op,4100 Island Blvd #2301,Aventura,FL,33160.0,1800000.0,3.0,3.0,BELLINI AT WILLIAMS ISLAN,2235.0,NaN,2013.0,NaN,805.0,2811.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/4100-Island...,MARMLS,A11608468,N,Y,25.941140,-80.135967,https://www.redfin.com/FL/Aventura/4100-Island...,Nina Vazquez,The Keyes Company,Alfredo Morejon,The Keyes Company,Joanne Perez,"Williams Island Realty, LLC",None,None,6,blue,BELLINI AT WILLIAMS ISLAN at 4100 Island Blvd ...
6,PAST SALE,2024-11-18,Condo/Co-op,6000 Island Blvd #1804,Aventura,FL,33160.0,1800000.0,3.0,2.5,BELLAMARE @ WILLIAMS ISLA,2030.0,NaN,2005.0,NaN,887.0,2844.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11671839,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Karen Matluck,"Compass Florida, LLC",None,None,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2024-11-21,Condo/Co-op,7930 East Dr #1808,North Bay Village,FL,33141.0,1688000.0,5.0,3.5,ELOQUENCE ON THE BAY COND,2748.0,NaN,2009.0,NaN,614.0,2983.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11534963,N,Y,25.8

In [83]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: OCEANA BAL HARBOUR CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,600,000M to $6,562,500M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Coral Gables, Miami, Aventura, Miami Beach, North Bay Village, Bal Harbour, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 113 condo sales totaling $77,744,235 million from Nov. 17th to Nov. 24th. The previous week, brokers closed 102 condo sales totaling $92,500,000.

Last week’s units sold for an average of $688,002, lower than the $1,022,796 average price fro

In [91]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Bal-Harbour/10201-Collins-Ave-33154/unit-2406/home/56744211


In [99]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/1000-S-Pointe-Dr-33139/unit-2503/home/43259346


In [93]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/1451-Brickell-Ave-33131/unit-3004/home/56742989


In [94]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Coral-Gables/13647-Deering-Bay-Dr-33158/unit-112/home/42815126


In [95]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Bal-Harbour/10201-Collins-Ave-33154/unit-2406/home/56744211


In [101]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/13647-Deering-Bay-Dr-33158/unit-112/home/42815126


In [102]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",1,orange,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
1,PAST SALE,2024-11-22,Condo/Co-op,1000 S Pointe Dr #2503,Miami Beach,FL,33139.0,3850000.0,3.0,2.5,MURANO AT PORTOFINO CONDO,2008.0,NaN,2002.0,NaN,1917.0,2782.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11643530,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Michelle Halfon,Sator Realty LLC,None,None,Laura Pantano,Coldwell Banker Realty,None,None,2,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
2,PAST SALE,2024-11-19,Condo/Co-op,10201 Collins Ave #905,Bal Harbour,FL,33154.0,3450000.0,2.0,2.5,OCEANA BAL HARBOUR CONDO,1704.0,NaN,2016.0,NaN,2025.0,3655.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11649870,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Allen Davoudpour,"Compass Florida, LLC",None,None,None,None,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2024-11-20,Condo/Co-op,1000 S Pointe Dr #705,Miami Beach,FL,33139.0,2600000.0,2.0,2.5,MURANO AT PORTOFINO,1407.0,NaN,2002.0,NaN,1848.0,1967.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11568588,N,Y,25.768527,-80.137591,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Lisa Van Wagenen,Brown Harris Stevens,Gabriela Pellegrini,Brown Harris Stevens,Jim Brinker,LoKation,None,None,4,blue,MURANO AT PORTOFINO at 1000 S Pointe Dr #705 i...
4,PAST SALE,2024-11-20,Condo/Co-op,296 Sea View Dr Unit C4,Key Biscayne,FL,33149.0,2160000.0,3.0,3.5,CAPE FLA CLUB CONDO CLUST,1800.0,NaN,1970.0,NaN,1200.0,1927.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,MARMLS,A11666942,N,Y,25.685372,-80.158768,https://www.redfin.com/FL/Key-Biscayne/296-Sea...,Sandra Poli,United Realty Group Inc,None,None,Ignacio Villanueva,Douglas Elliman,None,None,5,blue,CAPE FLA CLUB CONDO CLUST at 296 Sea View Dr U...
5,PAST SALE,2024-11-19,Condo/Co-op,4100 Island Blvd #2301,Aventura,FL,33160.0,1800000.0,3.0,3.0,BELLINI AT WILLIAMS ISLAN,2235.0,NaN,2013.0,NaN,805.0,2811.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/4100-Island...,MARMLS,A11608468,N,Y,25.941140,-80.135967,https://www.redfin.com/FL/Aventura/4100-Island...,Nina Vazquez,The Keyes Company,Alfredo Morejon,The Keyes Company,Joanne Perez,"Williams Island Realty, LLC",None,None,6,blue,BELLINI AT WILLIAMS ISLAN at 4100 Island Blvd ...
6,PAST SALE,2024-11-18,Condo/Co-op,6000 Island Blvd #1804,Aventura,FL,33160.0,1800000.0,3.0,2.5,BELLAMARE @ WILLIAMS ISLA,2030.0,NaN,2005.0,NaN,887.0,2844.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11671839,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Karen Matluck,"Compass Florida, LLC",None,None,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2024-11-21,Condo/Co-op,7930 East Dr #1808,North Bay Village,FL,33141.0,1688000.0,5.0,3.5,ELOQUENCE ON THE BAY COND,2748.0,NaN,2009.0,NaN,614.0,2983.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11534963,N,Y,25.8

## Time on Market Calculator

In [100]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 18) ## List (Earlier) date
date2 = datetime(2024, 11, 22) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

96


## Clean CSV for Datawrapper Chart

In [103]:
chart_df = df_top_ten

In [104]:
chart_df = chart_df.fillna(" ")

In [105]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [106]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [107]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [108]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [109]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [110]:
csv_date_string = today.strftime("%m_%d_%Y")

In [111]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [112]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-18,10201 Collins Ave #2406 in Bal Harbour,"$6,562,500",2,3.5,"2,625",2016,"$2,500",MARMLS,A11559220,"Chad Carroll Compass Florida, LLC","Chad Carroll Compass Florida, LLC Chad Carroll..."
1,2024-11-22,1000 S Pointe Dr #2503 in Miami Beach,"$3,850,000",3,2.5,"2,008",2002,"$1,917",MARMLS,A11643530,Michelle Halfon Sator Realty LLC,Laura Pantano Coldwell Banker Realty
2,2024-11-19,10201 Collins Ave #905 in Bal Harbour,"$3,450,000",2,2.5,"1,704",2016,"$2,025",MARMLS,A11649870,"Allen Davoudpour Compass Florida, LLC",
3,2024-11-20,1000 S Pointe Dr #705 in Miami Beach,"$2,600,000",2,2.5,"1,407",2002,"$1,848",MARMLS,A11568588,Lisa Van Wagenen Brown Harris Stevens Gabriela...,Jim Brinker LoKation
4,2024-11-20,296 Sea View Dr Unit C4 in Key Biscayne,"$2,160,000",3,3.5,"1,800",1970,"$1,200",MARMLS,A11666942,Sandra Poli United Realty Group Inc,Ignacio Villanueva Douglas Elliman
5,2024-11-19,4100 Island Blvd #2301 in Aventura,"$1,800,000",3,3.0,"2,235",2013,$805,MARMLS,A11608468,Nina Vazquez The Keyes Company Alfredo Morejon...,"Joanne Perez Williams Island Realty, LLC"
6,2024-11-18,6000 Island Blvd #1804 in Aventura,"$1,800,000",3,2.5,"2,030",2005,$887,MARMLS,A11671839,"Karen Matluck Compass Florida, LLC",Richard Goihman Douglas Elliman
7,2024-11-21,7930 East Dr #1808 in North Bay Village,"$1,688,000",5,3.5,"2,748",2009,$614,MARMLS,A11534963,Sebastien Studer Studer Realty LLC,Belkis Jorge Brown Harris Stevens
8,2024-11-22,13647 Deering Bay Dr #112 in Coral Gables,"$1,625,000",3,3.5,"3,140",1993,$518,MARMLS,A11541336,,
9,2024-11-22,1451 Brickell Ave #3004 in Miami,"$1,600,000",2,2.5,"1,437",2017,"$1,113",MARMLS,A11620882,"Nancy Batchelor Compass Florida, LLC. Michelle...","Ekaterina Sizenova Big International Realty, I..."
